## Testes da API

Nesse notebook são feitos alguns testes com a API do Olho Vivo

In [1]:
import pandas as pd
from requests import Session
from typing import Optional

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

#espera = 30
for i in range(1):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    #time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '13:45',
  'l': [{'c': '917H-10',
    'cl': 33224,
    'sl': 2,
    'lt0': 'METRÔ VL. MARIANA',
    'lt1': 'TERM. PIRITUBA',
    'qv': 14,
    'vs': [{'p': 11897,
      'a': True,
      'ta': '2025-09-16T16:45:11Z',
      'py': -23.53529425,
      'px': -46.71350125,
      'sv': None,
      'is': None},
     {'p': 11854,
      'a': True,
      'ta': '2025-09-16T16:45:09Z',
      'py': -23.56325125,
      'px': -46.654227000000006,
      'sv': None,
      'is': None},
     {'p': 11896,
      'a': True,
      'ta': '2025-09-16T16:45:12Z',
      'py': -23.4858895,
      'px': -46.739630500000004,
      'sv': None,
      'is': None},
     {'p': 11904,
      'a': True,
      'ta': '2025-09-16T16:45:22Z',
      'py': -23.579057249999998,
      'px': -46.639790500000004,
      'sv': None,
      'is': None},
     {'p': 11899,
      'a': True,
      'ta': '2025-09-16T16:45:03Z',
      'py': -23.551383,
      'px': -46.676852999999994,
      'sv': None,
      'is': None},
     {'p': 1190

In [6]:
len(posicoes)

1

In [7]:
rows = []

for h in posicoes:
    if h:
        for l in h['l']:

            for veiculo in l['vs']:

                row = {
                    'linha': l['c'],
                    'id_onibus': veiculo['p'],
                    'timestamp': veiculo['ta'],
                    'lat': veiculo['py'],
                    'lon': veiculo['px'] 
                }

                rows.append(row)

df_posicoes = pd.DataFrame(rows)

Conversão de horário

In [8]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,917H-10,11897,2025-09-16T16:45:11Z,-23.535294,-46.713501
1,917H-10,11854,2025-09-16T16:45:09Z,-23.563251,-46.654227
2,917H-10,11896,2025-09-16T16:45:12Z,-23.485889,-46.739631
3,917H-10,11904,2025-09-16T16:45:22Z,-23.579057,-46.639791
4,917H-10,11899,2025-09-16T16:45:03Z,-23.551383,-46.676853
...,...,...,...,...,...
9805,6020-10,66902,2025-09-16T16:45:35Z,-23.826263,-46.744140
9806,1775-10,26466,2025-09-16T16:45:08Z,-23.460980,-46.628096
9807,4008-21,48384,2025-09-16T16:45:19Z,-23.564962,-46.443078
9808,8545-10,13547,2025-09-16T16:45:25Z,-23.465033,-46.695086


In [9]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

df_posicoes['timestamp'] = df_posicoes['timestamp'].dt.tz_convert("America/Sao_Paulo")

df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,917H-10,11897,2025-09-16 13:45:11-03:00,-23.535294,-46.713501
1,917H-10,11854,2025-09-16 13:45:09-03:00,-23.563251,-46.654227
2,917H-10,11896,2025-09-16 13:45:12-03:00,-23.485889,-46.739631
3,917H-10,11904,2025-09-16 13:45:22-03:00,-23.579057,-46.639791
4,917H-10,11899,2025-09-16 13:45:03-03:00,-23.551383,-46.676853
...,...,...,...,...,...
9805,6020-10,66902,2025-09-16 13:45:35-03:00,-23.826263,-46.744140
9806,1775-10,26466,2025-09-16 13:45:08-03:00,-23.460980,-46.628096
9807,4008-21,48384,2025-09-16 13:45:19-03:00,-23.564962,-46.443078
9808,8545-10,13547,2025-09-16 13:45:25-03:00,-23.465033,-46.695086


In [10]:
horas = df_posicoes["timestamp"].dt.hour

horas_unicas = horas.unique()
horas_unicas.sort()

print(horas_unicas)

[13]


Ônibus duplicados

In [11]:
duplicados = (df_posicoes.groupby(["id_onibus", "timestamp"]).filter(lambda x: len(x) > 1).sort_values(["id_onibus", "timestamp"]))

duplicados

,linha,id_onibus,timestamp,lat,lon
